In [1]:
import argparse
import os
import numpy as np
import time
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt

from dataset import *
from model import *
from utils import *
from scipy.special import softmax
class_num = 4 #cat dog person background

num_epochs = 30
batch_size = 32


boxs_default = default_box_generator([10,5,3,1], [0.2,0.4,0.6,0.8], [0.1,0.3,0.5,0.7])


#Create network
network = SSD(class_num)
network.cuda()
cudnn.benchmark = True

/home/mka219/.conda/envs/image_segmentation/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_test = COCO("data/test/images/", "data/test/annotations/", class_num, boxs_default, train = "test", image_size=320)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=True, num_workers=0)

inference


In [3]:
network.load_state_dict(torch.load('network_3feb.pth'))
network.eval()
len(dataloader_test)

711

In [4]:
final_bound_box_class=[]

for i, data in enumerate(dataloader_test, 0):
    images_, filename= data
    images = images_.cuda()
    
    pred_confidence, pred_box = network(images)

    pred_confidence_ = pred_confidence[0].detach().cpu().numpy()
    pred_box_ = pred_box[0].detach().cpu().numpy()
    
    pred_confidence_,pred_box_, ogn_pred_box = non_maximum_suppression(pred_confidence_,pred_box_,boxs_default)
    
    #TODO: save predicted bounding boxes and classes to a txt file.
    #you will need to submit those files for grading this assignment
    #print(filename[0])
    
    visualize_pred("inference", pred_confidence_, pred_box_, [], [], images_[0].numpy(), boxs_default, ogn_pred_box, final_bound_box_class)
    #print(final_bound_box_class[i])
    final_bound_box_class[i].insert(0,filename[0])
    #print(final_bound_box_class)
    #if i==20:
    #    break


In [5]:
with open('predictions.txt', 'w') as f:
    for line in final_bound_box_class:
        f.write(f"{line}\n")

67